In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image

In [2]:
# Helper functions

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

In [3]:
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(20, len(files)) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])
print(len(gt_imgs)) 
n = 10 # Only use 10 images for training

Loading 20 images
satImage_001.png
Loading 20 images
satImage_001.png
20


In [4]:
patch_size = 16 # each patch is 16*16 pixels
img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]
# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [5]:
# Extract 6-dimensional features consisting of average RGB color as well as variance
def extract_features(img):
    feat_m = np.mean(img, axis=(0,1))
    feat_v = np.var(img, axis=(0,1))
    feat = np.append(feat_m, feat_v)
    return feat

# Extract 2-dimensional features consisting of average gray color as well as variance
def extract_features_2d(img):
    feat_m = np.mean(img)
    feat_v = np.var(img)
    feat = np.append(feat_m, feat_v)
    return feat

# Extract features for a given image
def extract_img_features(filename):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size)
    X = np.asarray([ extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
    return X

In [6]:
# Compute features for each image patch
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0

X = np.asarray([ extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

In [7]:
# train a logistic regression classifier
from sklearn import linear_model
# we create an instance of the classifier and fit the data
logreg = linear_model.LogisticRegression(C=1e5, class_weight="balanced")
logreg.fit(X, Y)

LogisticRegression(C=100000.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [15]:
# Use classifier on test set
# Loaded a set of images
root_dir = "test_set_images-2/"
files=os.listdir(root_dir)
n_te=len(files)
imgs_te=[]
for i in range(1,n_te):
    dir_img=root_dir+files[i]+'/'
    file=os.listdir(dir_img)
    imgs_te.append(load_image(dir_img+file[0]))

In [16]:
root_dir = "test_set_images-2/"
files=os.listdir(root_dir)

In [17]:
files

['.DS_Store',
 'test_001',
 'test_002',
 'test_003',
 'test_004',
 'test_005',
 'test_006',
 'test_007',
 'test_008',
 'test_009',
 'test_010',
 'test_011',
 'test_012',
 'test_013',
 'test_014',
 'test_015',
 'test_016',
 'test_017',
 'test_018',
 'test_019',
 'test_020',
 'test_021',
 'test_022',
 'test_023',
 'test_024',
 'test_025',
 'test_026',
 'test_027',
 'test_028',
 'test_029',
 'test_030',
 'test_031',
 'test_032',
 'test_033',
 'test_034',
 'test_035',
 'test_036',
 'test_037',
 'test_038',
 'test_039',
 'test_040',
 'test_041',
 'test_042',
 'test_043',
 'test_044',
 'test_045',
 'test_046',
 'test_047',
 'test_048',
 'test_049',
 'test_050']

In [18]:
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels
img_patches_te = [img_crop(imgs_te[i], patch_size, patch_size) for i in range(n_te-1)]
# Linearize list of patches
img_patches_te = np.asarray([img_patches_te[i][j] for i in range(len(img_patches_te)) for j in range(len(img_patches_te[i]))])
X_te = np.asarray([ extract_features_2d(img_patches_te[i]) for i in range(len(img_patches_te))])

In [19]:
# Predict on the training set
Z = logreg.predict(X_te)
Z.shape

(72200,)

In [20]:
# Convert array of labels to an image
def label_to_img(imgwidth, imgheight, w, h, labels):
    im = np.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            im[j:j+w, i:i+h] = labels[idx]
            idx = idx + 1
    return im

In [21]:
for img_idx in range(n_te-1):
    imgwidth = imgs_te[img_idx].shape[0]
    imgheight = imgs_te[img_idx].shape[1]
    im = np.zeros([imgwidth, imgheight], dtype=np.uint8)
    w = int(patch_size)
    h = int(patch_size)
    Z_img=Z[1444*img_idx:1444*(img_idx+1)]*255
    idx=0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            im[j:j+w, i:i+h] = Z_img[idx]
            idx=idx+1
    str_img_idx=str(img_idx+1)
    img_te = Image.fromarray(im)
    img_te.save("test_"+str_img_idx+".png")

In [22]:
import re
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

In [23]:
def patch_to_label(patch):
    df = np.mean(patch)
    if df > foreground_threshold:
        return 1
    else:
        return 0


def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\d+", image_filename).group(0))
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))



In [24]:
if __name__ == '__main__':
    submission_filename = 'dummy_submission1.csv'
    image_filenames = []
    for i in range(50):
        ii=str(i+1)
        image_filename = 'test_' + ii + '.png'
        image_filenames.append(image_filename)
    masks_to_submission(submission_filename, *image_filenames)